In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from plotnine import *

In [2]:
ny=pd.read_csv('data/NYPD_Motor_Vehicle_Collisions.csv')
la=pd.read_csv('data/LA_Traffic_Collision_Data_from_2010_to_Present.csv')
ia=pd.read_csv('data/Iowa_Crash_Data_Location.csv')
ny=ny.iloc[:1000,:]
la=la.iloc[:1000,:]
ia=ia.iloc[:1000,:]

/home/kchuangk/.pyenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (3,9,22,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
mo=pd.read_csv('data/mo_codes.csv')

In [76]:

def formatTime(dforig,location='ny'):
    def convertLATime(t):
        t=str(t)
        if len(t) == 1:
            return '00:0' + t
        elif len(t) == 2:
            return '00:' + t
        elif len(t)==3:
            return t[0]+":"+t[1:]
        else:
            return t[:1]+":"+t[2:]

    df=dforig.copy() 
    if location=='ny':
        df['date']=pd.to_datetime(df.DATE+ ' ' + df.TIME)
                #+ pd.to_timedelta(ny.TIME.map(lambda x: x.split(':')[1]),unit='minutes')
        return df
    elif location=='la':
        df['date']=pd.to_datetime(df['Date Occurred'] + ' ' + df['Time Occurred'].map(lambda x: convertLATime(x )))
        return df
    elif location=='ia':
        df['date']=pd.to_datetime(df['CRASH_DATE'].map(lambda x: re.match("(.*)(T)",x)[1]) + ' ' + df['TIMESTR'])
        return df

def mapLA_MO_Codes(x,mo_df=mo):
    x=str(x)
    x=x.split(' ')
    l=[]
    for i in x:
        if i=='nan' or (int(i)>=947 and int(i)<=999):
            i=-1
            l.append('NaN')
        else:
            crit=mo['MO_Code']==int(i)
            l.append(mo.loc[mo['MO_Code']==int(i),'MO_Description'].values[0])
    return l

    
ny_tmp=formatTime(ny,'ny')
la_tmp=formatTime(la,'la')
la_tmp['MO Codes']=la_tmp['MO Codes'].map(mapLA_MO_Codes)
ia_tmp=formatTime(ia,'ia')

In [79]:
la_tmp.tail().T

,995,996,997,998,999
DR Number,190810155,190810156,190810174,191110980,191214473
Date Reported,05/23/2019,05/23/2019,05/23/2019,05/24/2019,05/24/2019
Date Occurred,05/23/2019,05/23/2019,05/23/2019,05/23/2019,05/23/2019
Time Occurred,1530,1531,945,1420,1240
Area ID,8,8,8,11,12
Area Name,West LA,West LA,West LA,Northeast,77th Street
Reporting District,855,855,828,1124,1256
Crime Code,997,997,997,997,997
Crime Code Description,TRAFFIC COLLISION,TRAFFIC COLLISION,TRAFFIC COLLISION,TRAFFIC COLLISION,TRAFFIC COLLISION
MO Codes,"[T/C - Veh vs Veh, T/C - (C) Complaint of Inju...","[T/C - Veh vs Veh, T/C - (C) Complaint of Inju...","[T/C - Veh vs Veh, T/C - (B) Visible Injury, T...","[T/C - PCF (A) In the Narrative, T/C - Type of...",[NaN]


'nan'

In [294]:
la.loc[970,'MO Codes']

'3101 3401 3701 3004 3030 0605'

In [59]:
mo.loc[mo.MO_Code==int('0605'),'MO_Description'].tolist()[0]

'Traffic Accident/Traffic related incident'

In [43]:
mo.tail()

,MO_Code,MO_Description
772,605,T/C - Topanga
773,605,T/C - Central Traffic (CTD)
774,605,T/C - South Traffic (STD)
775,605,T/C - Valley Traffic (VTD)
776,605,T/C - West Traffic (WTD)
